<img style="float: left;padding: 1.2em" src="https://github.com/FrancescoIacovelli/XIII_ET_Symposium_Hakathon/blob/main/ET_Logo.png">  

#  XIII ET Symposium Hakathon session


#### Tutorial on ``gwfast``

This notebook is a simple tutorial to start playing around with the ``gwfast`` software, which is suitable for SNR and Fisher-matrix based parameter estimation forecasts on large catalogues of events.



## Installation for Google Colab

In [ ]:
#! pip install -q 'git+https://github.com/CosmoStatGW/gwfast' 

**Note**: Using Google Colab, you need to restart the kernel runtime after installation.

## Now import some packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py

import copy
from astropy.cosmology import Planck18

In [2]:
from gwfast import gwfastGlobals as glob

from gwfast.waveforms import IMRPhenomD_NRTidalv2
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from gwfast.fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams
from gwfast import gwfastUtils as utils

## COMPLETE EXAMPLE: GW170817 @ ET!

###  We use the Sardinia site for definiteness

In [3]:
alldetectors = copy.deepcopy(glob.detectors)
print('All available detectors are: '+str(list(alldetectors.keys())))

# select only ET in Sardinia
ETSdet = {det:alldetectors[det] for det in ['ETS']}
print('Using detector '+str(list(ETSdet.keys())))


All available detectors are: ['L1', 'H1', 'Virgo', 'KAGRA', 'LIGOI', 'ETS', 'ETMR', 'CE1Id', 'CE2NM', 'CE2NSW']
Using detectors ['L1', 'H1', 'Virgo']


In [4]:
# We use the ET-D psds
ETSdet['ETS']['psd_path'] = os.path.join(glob.detPath, 'ET-0000A-18.txt')

### Initialise the signals and then the network 
(in this case we have a single triangular detector, but we will see later why this syntax is useful)

In [ ]:
mywf = IMRPhenomD_NRTidalv2(fRef=20.)

mySignals = {}

for d in ETSdet.keys():

    mySignals[d] = GWSignal(mywf, 
                psd_path=ETSdet[d]['psd_path'],
                detector_shape = ETSdet[d]['shape'],
                det_lat= ETSdet[d]['lat'],
                det_long=ETSdet[d]['long'],
                det_xax=ETSdet[d]['xax'], 
                verbose=False,
                useEarthMotion = True,
                fmin=2.) 
        
myNet = DetNet(mySignals, verbose=False)      

Using ASD from file /Users/francesco.iacovelli/Desktop/PhD/Research/2023-04_GWFAST/gwfast/psds/LVC_O1O2O3/2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8
Using ASD from file /Users/francesco.iacovelli/Desktop/PhD/Research/2023-04_GWFAST/gwfast/psds/LVC_O1O2O3/2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8
Using ASD from file /Users/francesco.iacovelli/Desktop/PhD/Research/2023-04_GWFAST/gwfast/psds/LVC_O1O2O3/Hrec_hoft_V1O2Repro2A_16384Hz.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8


### Now we build a dictionary containing the parameters of GW170817

In [7]:
# Median values of the posterior samples for all the parameters, 
# except psi and the coalescence phase that are set to 0

z = np.array([0.00980])
tGPS = np.array([1187008882.4])

GW170817 = {'Mc':np.array([1.1859])*(1.+z), 
            'dL':Planck18.luminosity_distance(z).value/1000., 
            'theta':np.array([np.pi/2. + 0.4080839999999999]), 
            'phi':np.array([3.4461599999999994]),
            'iota':np.array([2.545065595974997]), 
            'psi':np.array([0.]),
            'tcoal':utils.GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0° 
            'eta':np.array([0.24786618323504223]), 
            'Phicoal':np.array([0.]), 
            'chi1z':np.array([0.005136138323169717]), 
            'chi2z':np.array([0.003235146993487445]), 
            'Lambda1':np.array([368.17802383555687]), 
            'Lambda2':np.array([586.5487031450857])
           }

print('Parameters for GW170817 are:')
GW170817

Parameters for GW170817 are:


{'Mc': array([1.19752182]),
 'dL': array([0.04374755]),
 'theta': array([1.97888033]),
 'phi': array([3.44616]),
 'iota': array([2.5450656]),
 'psi': array([0.]),
 'tcoal': DeviceArray([0.43432288], dtype=float64),
 'eta': array([0.24786618]),
 'Phicoal': array([0.]),
 'chi1z': array([0.00513614]),
 'chi2z': array([0.00323515]),
 'Lambda1': array([368.17802384]),
 'Lambda2': array([586.54870315])}

### Let's see how the signal looks like at one of the interferometers

In [ ]:
# first define the frequency grid
fmax = mywf.fcut(**GW170817)
fgrid = np.geomspace(2.,fmax,10000)

AmplatET_p, AmplatET_c = myNet.signals['ETS'].GWAmplitudes(GW170817, fgrid)

### Compute the expected matched-filter SNR

In [8]:
SNR = myNet.SNR(GW170817)
print('SNR for GW170817 at ET is %.2f'%SNR[0])

SNR for GW170817 is 33.16 to compare with 33


### Compute the total Fisher matrix

In [9]:
totF = myNet.FisherMatr(GW170817)
print('The computed Fisher matrix has shape %s'%str(totF.shape))

Computing Fisher for L1...
Computing derivatives...
Computing Fisher for H1...
Computing derivatives...
Computing Fisher for Virgo...
Computing derivatives...
Done.
The computed Fisher matrix has shape (13, 13, 1)


In [10]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = mywf.ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))


The relative difference is 2.22e-16 !


### Compute the covariance and perform some checks

In [11]:
totCov, inversion_err = CovMatr(totF)


In [12]:
_ = check_covariance(totF, totCov)


Inversion errors: [5657.625]
diagonal-1 = [array([-7.49594165e-09,  5.77928176e-08, -4.95498282e-14,  9.54375468e-14,
        6.81787959e-13,  2.98029830e-13,  8.52979909e-11,  1.76168555e-08,
        1.16116894e-09, -5.50993718e-06, -4.76458808e-06, -4.77896833e-10,
       -5.41731424e-08], dtype=float128)]
Max off diagonal: [5657.625]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False],
       [False,  True, False,  True,  True, False,  True,  True, False,
         True,  True, False, False],
       [False,  True,  True, False, False, False, False,  True, False,
         True, False, False, False],
       [False,  True, False,  True, False, False, False, False, False,
        False, False, False, False],
       [False,  True, False, False,  True, False, False, False, False,
        False, False, False, False],
       [ True, False, False, False, False,  

### Now try to eliminate the row corresponding to $\delta\tilde{\Lambda}$, and see that the inversion error lowers

In [13]:
ParNums = mywf.ParNums

newFish, newPars = fixParams(totF, ParNums, ['deltaLambda'])

print('Now the Fisher matrix has shape %s'%str(newFish.shape))

newCov, new_inversion_err = CovMatr(newFish)

_ = check_covariance(newFish, newCov)


Now the Fisher matrix has shape (12, 12, 1)
Inversion errors: [0.00099936]
diagonal-1 = [array([-2.28188846e-09, -2.06082540e-09,  1.96557180e-13,  1.75944762e-13,
       -9.95434637e-13,  2.58760678e-13, -9.65491662e-12,  4.94747979e-10,
        6.65625888e-12, -1.89773989e-07,  7.42870654e-09,  3.38172546e-12],
      dtype=float128)]
Max off diagonal: [1.3847575246472842991e-05]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True,  True, False, False, False, False, False,  True, False,
         True, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False,  True, False, False, False, False,
        False, False, False],
       [ True, False, False, False

### Finally compute the localisation region

In [14]:
skyArea = compute_localization_region(newCov, newPars, GW170817['theta'])
print('The estimated sky location is %.1f deg^2'%skyArea)


The estimated sky location is 19.9 deg^2, to compare with 16 deg^2
